# Projeto 2 - Ciência dos Dados

Nome: Pedro Braga Carani

Nome: Gustavo Rodrigues Pazemeckas

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [21]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [22]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [23]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [24]:
#Produto escolhido:
produto = 'fone jbl'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [25]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [26]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

No projeto o tweet foi classificado como relevante quando ele possui uma crítica ao produto seja ela positiva ou negativa. Logo, quando não houve crítica, o tweet foi considerado irrelevante.

Foi classificado 300 tweets no treinamento. 


___
### Montando o Classificador Naive-Bayes

   Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.
    
   Antes da montagem do Classificador, utilizamos a função "cleanup" para limpar os caracteres que podem atrapalhar na leitura dos dados. 
   

### Função Cleanup


In [27]:
import re 

def cleanup(text):

    punctuation = '[!\-.:?;#/,\n https]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

### Lendo o arquivo Treinamento

In [28]:
fone_Treinamento=pd.read_excel("fone jbl.xlsx", sheet_name="Treinamento")



### Preparando as listas de frequências das palavras (relevantes e irrelevantes)

In [29]:
#Relevantes
fone_r=fone_Treinamento[fone_Treinamento["classificacao"]==1]
textor="".join(fone_r["Treinamento"])
texto_cleanr=cleanup(textor)
lista_r=texto_cleanr.split()
freq_r=pd.Series(lista_r).value_counts(True)
#freq_r


#Irrelevantes
fone_ir=fone_Treinamento[fone_Treinamento["classificacao"]==0]
textoi="".join(fone_ir["Treinamento"])
texto_cleani=cleanup(textoi)
lista_ir=texto_cleani.split()
freq_ir=pd.Series(lista_ir).value_counts(True)
#freq_ir

soma_r=len(freq_r)
soma_ir=len(freq_ir)

total=soma_ir+soma_r

# Classificador Naive-Bayes

A proposta do Classificador é analisar as probalidades de um tweet ser relevante ou não dado que a frequência de cada palavra que está presente nesse mesmo tweet. Para isso, foi utilizado a tabela de frequências das palavras relevantes ou irrelevantes.

In [30]:
#Dados empiricos
prelevante=0.69
pirrelevante=0.31


def Naive_Bayes_Treinamento(tweet):


    frase=cleanup(tweet)
    frase1=frase.split()

    #P(Frase|Relevante)
    Pfraserelevante=freq_r.loc[frase1].prod()
    #P(Frase|Irrelevante)
    Pfraseirrelevante=freq_ir.loc[frase1].prod()
    
    #P(Relevante|Frase)    
    Prelevantefrase=Pfraserelevante*prelevante
                    
    #P(Irrelevante|Frase)
    Pirrelevantefrase=Pfraseirrelevante*pirrelevante

    if Prelevantefrase > Pirrelevantefrase:
        return 1 #Relevante
    
    else:
        return 0 #Irrelevante
    
#Exemplo
print (Naive_Bayes_Treinamento("Fone da Jbl é bom"))
                        

1


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  del sys.path[0]
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  from ipykernel import kernelapp as app


___
### Verificando a performance

Para a verificação da performance, foi utilizado os tweets da planilha Teste. 


### Lendo o arquivo Teste

In [31]:
fone_teste=pd.read_excel("fone jbl.xlsx", sheet_name="Teste")

### Classificador Naive-Bayes com Laplace

Como nem todas as palavras presentes no tweets "Teste" estavam presentes tambem nos tweets "Treinamento", foi necessário a utilização do algoritmo de Laplace smoothing. 

In [32]:

prelevante=0.69
pirrelevante=0.31
i=0
#Naive_Bayes com Laplace
for t in fone_teste["Teste"]:

    tweet=t
    frase=cleanup(tweet)
    frase1=frase.split()
    Prelevantefrase=0
    Pirrelevantefrase=0
    for p in frase1:
        try:  
            #P(Frase|Irrelevante)
            Pfraseirrelevante=freq_ir.loc[p].prod()

                    
            #P(Irrelevante|Frase)
            Pirrelevantefrase=Pfraseirrelevante*pirrelevante
        
            #P(Frase|Relevante)
            Pfraserelevante=freq_r.loc[p].prod()
            #P(Relevante|Frase)    
            Prelevantefrase=Pfraserelevante*prelevante

        except:

                    
            Prelevantefrase += 1/(soma_r + total)
            Pirrelevantefrase +=1/(soma_ir + total)


    if Prelevantefrase > Pirrelevantefrase:
        fone_teste["Verifica"][i]=1
        #Relevante
    
    else:
        fone_teste["Verifica"][i]=0
        #Irrelevante
    
    i+=1


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [33]:
fone_teste.head(5)

,Teste,Classificacao,Verifica
0,"eu nao tenho sorte com fone cara, nao deu 1 me...",1,1.0
1,ouvir música 8d no fone jbl e um caminho sem v...,1,1.0
2,amanhã vou tomar vergonha na cara e comprar um...,1,1.0
3,rt @kadashil: bom dia me dá um fone da jbl pfv,0,1.0
4,"fone de ouvido in-ear, jbl, = https://t.co/ojo...",0,0.0


In [34]:
Classificacao=fone_teste["Classificacao"]
Verifica=fone_teste["Verifica"]
pd.crosstab(Classificacao, Verifica)

Verifica,0.0,1.0
Classificacao,,
0,10,21
1,1,28


In [35]:
verdadeiro_positivos=0
falso_positivos=0
verdadeiro_negativos=0
falso_negativos=0
total=verdadeiro_positivos+falso_positivos+verdadeiro_negativos+falso_negativos

i=0
for t in fone_teste["Verifica"]:
    if fone_teste["Verifica"][i]== 1 and fone_teste["Classificacao"][i]==0:
        falso_positivos+=1
    if fone_teste["Verifica"][i]== 0 and fone_teste["Classificacao"][i]==0:
        verdadeiro_negativos+=1  
    if fone_teste["Verifica"][i]== 1 and fone_teste["Classificacao"][i]==1:
        verdadeiro_positivos+=1
    if fone_teste["Verifica"][i]== 0 and fone_teste["Classificacao"][i]==1:
        falso_negativos+=1
    i+=1 
total=verdadeiro_positivos+falso_positivos+verdadeiro_negativos+falso_negativos
porcentagem_falso_positivos=(falso_positivos/total)*100
porcentagem_verdadeiro_positivos=(verdadeiro_positivos/total)*100
porcentagem_falso_negativos=(falso_negativos/total)*100
porcentagem_verdadeiro_negativos=(verdadeiro_negativos/total)*100



In [36]:
print("Porcentagem de verdadeiros positivos: {}%". format(int(porcentagem_verdadeiro_positivos)))
print("Porcentagem de falsos positivos: {}%". format(int(porcentagem_falso_positivos)))
print("Porcentagem de verdadeiros negativos: {}%". format(int(porcentagem_verdadeiro_negativos)))
print("Porcentagem de falsos negativos: {}%". format(int(porcentagem_falso_negativos)))

Porcentagem de verdadeiros positivos: 46%
Porcentagem de falsos positivos: 35%
Porcentagem de verdadeiros negativos: 16%
Porcentagem de falsos negativos: 1%


___
### Concluindo

Após verificar a perfomance do Classificado Naives-Bayes com a base de testes foi possível observar que houve uma taxa de acerto relativamente alta, já que a base de treinamento não utiliza uma quantidade de tweets suficientes para uma análise mais consistente.
       
Sendo assim, uma possível melhoria para esse classificador seria um número maior de tweets analisados para uma precisão maior na tabela de frequências das palavras. Além disso, outra melhoria possível seria dividir os tweets em mais categorias, como por exemplo: bom, neutro ou ruim. Assim, a porcentagem de verdadeiros ou falsos positivos aumentará.
       
O algoritmo não pode ser realimentado pelos tweets já classificados, pois ainda há erros, e quando realimentado, diminuirá ainda mais a "precisão" do classificador.
       
Um dos possíveis cenários que o classificador pode ser útil é nos serviços de streaming, como por exemplo, na avaliação de um filme de um certo genêro dado que os outros filmes desse mesmo genêro possuem tal avalição.

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**